In [4]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../../../../')
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from data.engines.model import Record2
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list
from optimization.bayes_optimization_xgb import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

from data.engines.sqlengine import SQLEngine

weekly = 's1'  # s1, s2,  s3, s4, s5, s_1
record_engine = SQLEngine('sqlite:///./{}/real_tune_record_without_alpha.db'.format(weekly))


In [5]:
import datetime
# ref_date_pre = '2019-12-27'

previous_pos = record_engine.fetch_record_meta(Record2, trade_date=None)
previous_pos

,trade_date,weight,industry,er,code
0,2019-12-12,9.239730e-10,房地产,0.222482,6
1,2019-12-12,6.409436e-10,机械设备,0.222681,8
2,2019-12-12,6.771609e-10,综合,0.223521,9
3,2019-12-12,6.219948e-10,建筑材料,0.221045,12
4,2019-12-12,5.950755e-10,电子,0.222678,21
5,2019-12-12,6.208724e-10,汽车,0.222624,25
6,2019-12-12,8.383484e-10,公用事业,0.223786,27
7,2019-12-12,1.127531e-09,医药生物,0.226901,28
8,2019-12-12,9.550943e-10,房地产,0.223921,31
9,2019-12-12,6.463227e-10,机械设备,0.222296,39


In [6]:
type(previous_pos['trade_date'][0])

datetime.date

In [ ]:
record_engine.del_historical_data(Record2, '2020-01-10')  # 删除同日期的历史数据


In [ ]:
previous_pos = record_engine.fetch_record_meta(Record2, trade_date=None)
previous_pos